In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt


class MultiVariable:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.X_mean = np.mean(X, axis=0)
        self.X_std = np.std(X, axis=0)
        self.y_mean = np.mean(y)
        self.y_std = np.std(y)
        self.X = (X - self.X_mean) / self.X_std
        self.X = np.hstack((np.ones((self.X.shape[0], 1)), self.X))
        self.y = (y - self.y_mean) / self.y_std
        self.weights = np.zeros(self.X.shape[1])
        self.learning_rate = 0.001
        self.max_steps = 1000
        self.convergence_threshold = 1e-6
        self.round = None
        self.batch_size = 10
        self.n = X.shape[1]
        self.x_symbols = sp.symbols('x:{}'.format(self.n))

    def SoSR(self, weights = None):
        if weights is None:
            weights = self.weights
        SoSR = np.square(self.y - (weights @ self.X.T))
        return np.sum(SoSR)
    
    def gradient(self, X = None, y = None):
        if X is None:
            X = self.X
        if y is None:
            y = self.y
        return -2 * X.T @ (y - (self.weights @ X.T))

    def GradientDescent(self):
        print("\033[1mGradient Descent:\033[0m")
        losses = []
        prev_loss = 0
        for i in range(self.max_steps):
            weights_gradient = self.gradient()
            weights_temp = self.weights - self.learning_rate * weights_gradient
            loss = self.SoSR(weights_temp)
            losses.append(loss)
            if abs(loss - prev_loss) < self.convergence_threshold:
                print("Converged at iteration", i)
                break
            self.weights = weights_temp
            prev_loss = loss  

        self.show(losses)

    def MiniBatchGradientDescent(self):
        print("\033[1mMini-Batch Gradient Descent:\033[0m")
        losses = []
        prev_loss = 0
        for i in range(self.max_steps):
            for j in range(0, len(self.X), self.batch_size):
                X_batch = self.X[j : j + self.batch_size, : ]
                y_batch = self.y[j : j + self.batch_size]
                weights_gradient = self.gradient(X_batch, y_batch)
                weights_temp = self.weights - self.learning_rate * weights_gradient
                loss = self.SoSR(weights_temp)
                losses.append(loss)
                self.weights = weights_temp

            if abs(loss - prev_loss) < self.convergence_threshold:
                print("Converged at iteration", i)
                break
            prev_loss = loss

        self.show(losses)

    def StochasticGradientDescent(self):
        print("\033[1mStochastic Gradient Descent:\033[0m")
        losses = []
        prev_loss = 0
        for i in range(self.max_steps):
            for j in range(len(self.X)):
                x_point = self.X[j, :]
                y_point = self.y[j]
                weights_gradient = self.gradient(x_point[np.newaxis, :], np.array([y_point]))
                weights_temp = self.weights - self.learning_rate * weights_gradient
                loss = self.SoSR(weights_temp)
                losses.append(loss)
                self.weights = weights_temp

            if abs(loss - prev_loss) < self.convergence_threshold:
                print("Converged at iteration", i)
                break
            prev_loss = loss

        self.show(losses)

    def calculatePlaneEquation(self):
        weights = self.weights
        def plane_eq(x, y):
            return weights[0] + weights[1] * x + weights[2] * y
        return plane_eq

    def printEquation(self):
        weights = self.weights[1:] 
        bias = self.weights[0] 
        equation = bias
        for i in range(len(weights)):
            equation += weights[i] * (self.x_symbols[i] - self.X_mean[i]) / self.X_std[i]
        equation_expr = sum([w * (x - m) / s for w, x, m, s in zip(weights, self.x_symbols, self.X_mean, self.X_std)]) + bias
        equation_expr = sp.expand(equation_expr) * self.y_std + self.y_mean
        equation_expr_rounded = sp.nsimplify(equation_expr, rational=True).evalf()
        rounded_equation = sp.N(equation_expr_rounded, self.round) if self.round is not None else equation_expr_rounded
        print(sp.pretty(rounded_equation))

    def plotLoss(self, losses):
        plt.figure(figsize = (9, 6)) 
        plt.plot(range(len(losses)), losses, label = 'Loss function')
        plt.xlabel('Iteration')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.gca().set_aspect('auto', adjustable = 'box')
        plt.show()


    def plotResult(self, plane_equation):
        if self.X.shape[1] > 3:
            print("Plotting is supported only for data with up to three features.")
            return

        fig = plt.figure(figsize=(7, 7))
        ax = fig.add_subplot(111, projection = '3d')

        ax.scatter(self.X[:, 1], self.X[:, 2], self.y, c = self.y, cmap = 'viridis')

        xmin, xmax = np.min(self.X[:, 1]), np.max(self.X[:, 1])
        ymin, ymax = np.min(self.X[:, 2]), np.max(self.X[:, 2])
        xx, yy = np.meshgrid(np.linspace(xmin, xmax, 10), np.linspace(ymin, ymax, 10))
        
        zz = plane_equation(xx, yy)
        
        ax.plot_surface(xx, yy, zz, alpha = 0.5, cmap = 'viridis')

        ax.set_xlabel('X1')
        ax.set_ylabel('X2')
        ax.set_zlabel('Y')
        ax.set_title('Data and Regression Plane')
        plt.show()

    def show(self, losses):
        self.printEquation()
        plane_equation = self.calculatePlaneEquation()
        self.plotResult(plane_equation)
        self.plotLoss(losses)

